# Test evolving the $^{3}S_1$ channel with no coupling to $^{3}D_1$

__Author:__ A. J. Tropiano [atropiano@anl.gov]<br/>
__Date:__ March 7, 2023

This notebook tests the momentum distribution calculation in the $^{3}S_1$ channel with no coupling to $^{3}D_1$.

_Last update:_ March 14, 2023

In [1]:
# Python imports
import numpy as np
from scipy.integrate import ode
import time

In [2]:
# Imports from scripts
# from scripts.figures import set_rc_parameters
from scripts.potentials import Potential
from scripts.tools import replace_periods

In [3]:
# Run this cell to turn on customized matplotlib graphics
# set_rc_parameters()

## Set-up

In [4]:
# kvnn = 6  # AV18
kvnn = 111  # SMS N4LO 450 MeV
# channel = '1S0'
channel = '3S1'
kmax, kmid, ntot = 15.0, 3.0, 120
generator = 'Wegner'
lamb = 1.35

In [5]:
potential = Potential(kvnn, channel, kmax, kmid, ntot)

In [6]:
# Initial Hamiltonian with integration factors attached [MeV]
H_initial_weights = potential.load_hamiltonian()

## Set $^{3}S_1-^{3}D_1$ and $^{3}D_1-^{3}S_1$ off-diagonal blocks to zero

In [7]:
if channel == '3S1':
    H_initial_weights[:ntot, ntot:] = np.zeros((ntot, ntot))
    H_initial_weights[ntot:, :ntot] = np.zeros((ntot, ntot))

## SRG evolve the potential

Using `scipy.integrate.ode` and solving for $H(\lambda)$.

In [8]:
def matrix_to_vector(M):
    """
    Takes the upper triangle of the matrix M (including the diagonal) and
    reshapes it into a vector v.
        
    Parameters
    ----------
    M : 2-D ndarray
        Input matrix of shape (N, N).
            
    Returns
    -------
    v : 1-D ndarray
        Output vector of shape (N*(N+1)/2,).
            
    """

    # Length of matrix
    N = len(M)
    # Length of vectorized matrix
    n = int(N * (N + 1) / 2)

    # Initialize vectorized matrix
    v = np.zeros(n)

    # Algorithm for reshaping M to the vector v
    i = 0
    j = N
    for k in range(N):
        v[i:j] = M[k][k:]
        i = j
        j += N - k - 1

    return v

In [9]:
def vector_to_matrix(v):
    """
    Takes the vector of an upper triangle matrix v and returns the full matrix
    M. Use only for symmetric matrices.
        
    Parameters
    ----------
    v : 1-D ndarray
        Input vector of shape (N*(N+1)/2,).
        
    Returns
    -------
    output : 2-D ndarray
        Output matrix of shape (N, N).
            
    """
    
    # Dimension of matrix is found by solving for the positive solution to
    # n = N(N+1)/2, where n is the length of the vector
    N = int((-1 + np.sqrt(1 + 8 * len(v))) / 2)

    # Initialize matrix
    M = np.zeros((N, N))

    # Build the upper half of matrix with the diagonal included

    # Algorithm for reshaping v to the matrix M
    i = 0
    j = N
    for k in range(N):
        M[k, k:] = v[i:j]
        i = j
        j += N - k - 1

    # Now reflect the upper half to lower half to build full matrix
    # M.T - np.diag(np.diag(M)) is the lower half of M excluding diagonal
    return M + (M.T - np.diag(np.diag(M)))

In [10]:
def commutator(A, B):
    """Commutator of square matrices A and B."""

    return A @ B - B @ A

In [11]:
def eta(H_matrix):
    """Wegner generator \eta = [H_D, H]."""

    # G = H_D (diagonal of the evolving Hamiltonian)
    G_matrix = np.diag(np.diag(H_matrix))

    # \eta = [G, H]
    return commutator(G_matrix, H_matrix)

In [12]:
def H_deriv(lamb, H_vector):
    """Right-hand side of the SRG flow equation."""

    # Matrix form of the evolving Hamiltonian
    H_matrix = vector_to_matrix(H_vector)

    # Get SRG generator \eta = [G, H]
    eta_matrix = eta(H_matrix)

    # RHS of the flow equation in matrix form
    dH_matrix = -4.0 / lamb ** 5 * commutator(eta_matrix, H_matrix)

    # Returns vector form of RHS of flow equation
    dH_vector = matrix_to_vector(dH_matrix)

    return dH_vector

In [13]:
def select_step_size(solver_lambda, lambda_final):
    """Select ODE solver step-size depending on the extent of evolution. We
    can take bigger steps at large values of \lambda.
    """

    if solver_lambda >= 6.0:
        dlamb = 1.0
    elif 2.5 <= solver_lambda < 6.0:
        dlamb = 0.5
    elif 1.5 <= solver_lambda < 2.5:
        dlamb = 0.1
    else:
        dlamb = 0.05

    return dlamb

In [14]:
def get_ode_solver(lambda_initial, H_initial, atol, rtol):
    """Sets up the ODE solver."""

    # Solving for H(s)
    solver = ode(H_deriv)

    # Initial Hamiltonian as a vector
    H_initial = matrix_to_vector(H_initial)

    # Set initial conditions
    solver.set_initial_value(H_initial, lambda_initial)

    # Following the example in Hergert:2016iju with modifications to nsteps and
    # error tolerances
    solver.set_integrator('vode', method='bdf', order=5, atol=atol, rtol=rtol,
                          nsteps=5000000)

    return solver

In [15]:
def srg_evolve_wrt_lambda(
        H_initial_MeV, lambda_array, lambda_initial=20.0, atol=1e-10, rtol=1e-10
):
    """SRG evolve the Hamiltonian with respect to \lambda."""
    
    # Convert Hamiltonian from MeV to units [fm^-2]
    H_initial = H_initial_MeV / 41.47

    # Set-up ODE solver
    solver = get_ode_solver(lambda_initial, H_initial, atol, rtol)
    
    # Start time
    t0 = time.time()
    
    # Evolve the Hamiltonian to each value of \lambda and store in dictionary
    d = {}
    for lamb in lambda_array:

        # Solve ODE up to lamb and store in dictionary
        while solver.successful() and round(solver.t, 2) > lamb:
            
            # Get ODE solver step-size in \lambda
            dlamb = select_step_size(solver.t, lamb)
            
            # Integrate to next step in lambda
            solution_vector = solver.integrate(solver.t - dlamb)

        # Store evolved Hamiltonian matrix [MeV] in dictionary
        d[lamb] = vector_to_matrix(solution_vector) * 41.47

    # End time
    t1 = time.time()

    # Print details
    mins = round((t1 - t0) / 60.0, 4)  # Minutes elapsed evolving H(\lambda)
    print(f"Done evolving to \lambda = {lamb} fm^-1 after {mins:.4f} minutes.")

    return d

In [16]:
lambda_array = np.array([6.0, 3.0, 2.0, 1.5, 1.35])
d = srg_evolve_wrt_lambda(H_initial_weights, lambda_array, lambda_initial=20.0)

Done evolving to \lambda = 1.35 fm^-1 after 2.1610 minutes.


In [17]:
# Get H(\lambda=1.35)
H_evolved_weights = d[1.35]

## Save evolved Hamiltonian for `test_momentum_distribution_script.py`

In [18]:
def save_H_evolved(d, kvnn, channel, kmax, kmid, ntot, generator):

    for lamb in d:
            
        file_name = (
            f"H_evolved_kvnn_{kvnn}_{channel}_no_coupling_{generator}_lamb"
            f"_{lamb}_kmax_{kmax}_kmid_{kmid}_ntot_{ntot}_ode_BDF_wrt_lambda"
        )
            
        np.savetxt("./test_srg/" + replace_periods(file_name) + ".txt", d[lamb])

In [19]:
def load_H_no_coupling(kvnn, channel, generator, lamb, kmax, kmid, ntot):
    """Loads the evolved Hamiltonian assuming there are no coupled channels."""
    
    file_name = (
        f"H_evolved_kvnn_{kvnn}_{channel}_no_coupling_{generator}_lamb_{lamb}"
        f"_kmax_{kmax}_kmid_{kmid}_ntot_{ntot}_ode_BDF_wrt_lambda"
    )
    
    H_evolved = np.loadtxt("./test_srg/" + replace_periods(file_name) + ".txt")
    
    return H_evolved

In [20]:
save_H_evolved(d, kvnn, channel, kmax, kmid, ntot, generator)

In [21]:
# Test the loading function
H_temp = load_H_no_coupling(kvnn, channel, generator, lamb, kmax, kmid, ntot)

In [22]:
H_temp[:ntot,:ntot]

array([[ 5.80164057e-05, -1.59499112e-06, -4.90256750e-06, ...,
        -4.10199762e-14, -1.93200285e-13,  1.56839585e-10],
       [-1.59499112e-06,  1.60152762e-03, -3.93653508e-05, ...,
        -3.26350563e-13, -1.53708090e-12,  1.24779904e-09],
       [-4.90256750e-06, -3.93653508e-05,  9.60976870e-03, ...,
        -9.56181386e-13, -4.50352566e-12,  3.65595271e-09],
       ...,
       [-4.10199762e-14, -3.26350563e-13, -9.56181386e-13, ...,
         9.25451055e+03, -9.25068076e-02, -3.63951129e-02],
       [-1.93200285e-13, -1.53708090e-12, -4.50352566e-12, ...,
        -9.25068076e-02,  9.29962163e+03, -6.17799347e-02],
       [ 1.56839585e-10,  1.24779904e-09,  3.65595271e-09, ...,
        -3.63951129e-02, -6.17799347e-02,  9.32480957e+03]])

In [23]:
H_initial_weights[:ntot, :ntot]

array([[ 5.80373023e-05, -1.42719766e-06, -4.38676798e-06, ...,
         1.57892383e-18,  1.26395537e-18,  8.30019192e-19],
       [-1.42719766e-06,  1.60287495e-03, -3.52236461e-05, ...,
         1.26785588e-17,  1.01494019e-17,  6.66494920e-18],
       [-4.38676798e-06, -3.52236461e-05,  9.62250036e-03, ...,
         3.89789886e-17,  3.12033431e-17,  2.04907342e-17],
       ...,
       [ 1.57892383e-18,  1.26785588e-17,  3.89789886e-17, ...,
         9.25450931e+03, -1.33141418e-01, -8.71166415e-02],
       [ 1.26395537e-18,  1.01494019e-17,  3.12033431e-17, ...,
        -1.33141418e-01,  9.29962069e+03, -6.96606362e-02],
       [ 8.30019192e-19,  6.66494920e-18,  2.04907342e-17, ...,
        -8.71166415e-02, -6.96606362e-02,  9.32480915e+03]])

In [24]:
import numpy.linalg as la

eig_init, _ = la.eigh(H_initial_weights[:ntot, :ntot])
eig_evol, _ = la.eigh(H_temp[:ntot,:ntot])

for i, j in zip(eig_init, eig_evol):
    line = f"{i:.10e}\t{j:.10e}"
    print(line)

5.7443500172e-05	5.7443501374e-05
1.5653905541e-03	1.5653906279e-03
9.2771773710e-03	9.2771780512e-03
3.1384386642e-02	3.1384389630e-02
7.9259461210e-02	7.9259470005e-02
1.6741330010e-01	1.6741331991e-01
3.1369755887e-01	3.1369759546e-01
5.3956652626e-01	5.3956658442e-01
8.7011523245e-01	8.7011531511e-01
1.3337752606e+00	1.3337753687e+00
1.9617731587e+00	1.9617732920e+00
2.7875137351e+00	2.7875138925e+00
3.8459802130e+00	3.8459803928e+00
5.1731699629e+00	5.1731701631e+00
6.8055517413e+00	6.8055519593e+00
8.7795257822e+00	8.7795260149e+00
1.1130875089e+01	1.1130875333e+01
1.3894205678e+01	1.3894205928e+01
1.7102381011e+01	1.7102381262e+01
2.0785960082e+01	2.0785960326e+01
2.4972649865e+01	2.4972650092e+01
2.9686782245e+01	2.9686782444e+01
3.4948824097e+01	3.4948824253e+01
4.0774927565e+01	4.0774927662e+01
4.7176526312e+01	4.7176526333e+01
5.4159982652e+01	5.4159982582e+01
6.1726289846e+01	6.1726289680e+01
6.9870833889e+01	6.9870833633e+01
7.8583218770e+01	7.8583218445e+01
8.7847159257e+